In [1]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack #this line needs to be here in case we are running the GPU version of cuda in haystack this makes it run.
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-_wdd6diq/farm-haystack_45a21e7ad36844c8b20ee47af1ea4839
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-_wdd6diq/farm-haystack_45a21e7ad36844c8b20ee47af1ea4839
  Resolved https://github.com/deepset-ai/haystack.git to commit bb729ab95f12b63cbbeed2978d449544f855a82c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 15.1 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached tokenizers-0.12.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)


  Created wheel for farm-haystack: filename=farm_haystack-1.5.1rc0-py3-none-any.whl size=573383 sha256=3c73b4138e3c2c2bf3ebddaab4a9b09bdcee7e8172d8cdb624efc9d23d68e028
  Stored in directory: /tmp/pip-ephem-wheel-cache-4jv44pn8/wheels/97/0d/ae/77cde17929fbf66c8320f19b30789acfe52e2312bb1d125be1
Successfully built farm-haystack
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.13.0
    Uninstalling transformers-4.13.0:
      Successfully uninstalled transformers-4.13.0
  Attempting uninstall: farm-haystack
    Found existing installation: farm-haystack 1.4.0
    Uninstalling farm-haystack-1.4.0:
      Successfully uninstalled farm-haystack-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the 

In [2]:
from haystack.nodes import DensePassageRetriever
from haystack.utils import fetch_archive_from_http
from haystack.document_stores import InMemoryDocumentStore

INFO - haystack.document_stores.base -  Numba not found, replacing njit() with no-op implementation. Enable it with 'pip install numba'.
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [3]:
from haystack.nodes import PDFToTextConverter
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.nodes import FARMReader, ElasticsearchRetriever
from haystack.pipelines import ExtractiveQAPipeline, DocumentSearchPipeline
from haystack.nodes import PreProcessor
from haystack import Document

In [4]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [5]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_1"

In [6]:
retriever = DensePassageRetriever(
    document_store=InMemoryDocumentStore(),
    query_embedding_model=query_model,
    passage_embedding_model=passage_model,
    max_seq_len_query=64,
    max_seq_len_passage=400,
    use_gpu=False
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find IIC/dpr-spanish-question_encoder-allqa-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Automatically detected language from language model name: spanish
INFO - haystack.modeling.model.language_model -  Loaded IIC/dpr-spanish-question_encoder-allqa-base
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find IIC/dpr-spanish-passage_e

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=5,
    batch_size=5,
    grad_acc_steps=3,
    save_dir=save_dir,
    evaluate_every=3,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /w\   /w\   /w\   /w\   /|\  /w\   /|\  /w\   /w\   /w\   /|\  /w\   /w\   /w\   /|\
INFO - haystack.modeling.data_handler.data_silo -  /'\   /'\   /'\   / \   /'\   

INFO - haystack.modeling.evaluation.eval -  acc: 0.9093333333333333
INFO - haystack.modeling.evaluation.eval -  f1: 0.32
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.6146666666666667
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.76
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9514    0.9514    0.9514       350
     positive     0.3200    0.3200    0.3200        25

     accuracy                         0.9093       375
    macro avg     0.6357    0.6357    0.6357       375
 weighted avg     0.9093    0.9093    0.9093       375

Train epoch 0/4 (Cur. train loss: 10.7923):  57%|▌| 12/21 [01:56<01:21,  9.03s/i
Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.25s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
****************************************

Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.23s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 27 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.697548336982727
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.9146666666666666
INFO - haystack.modeling.evaluation.eval -  f1: 0.36
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.6373333333333333
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.6
INFO - haystack.modeling.

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.5593026542663574
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.92
INFO - haystack.modeling.evaluation.eval -  f1: 0.4000000000000001
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.66
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.28
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9571    0.9571    0.9571       350
     positive     0.4000    0.4000    0.4000        25

     accuracy                         0.9200       375
    macro avg     0.6786    0.6786    0.6786       375
 weighted avg     0.9200    0.9200    0.9200       375

Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.38s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//  

Train epoch 2/4 (Cur. train loss: 1.8386):  86%|▊| 18/21 [03:10<00:27,  9.16s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:10<00:00,  2.16s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 60 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.515546464920044
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.92
INFO - haystack.modeling.evaluation.eval -  f1: 0.4000000000000001
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.66
INFO - haystack.mode

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.5614055824279786
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.9253333333333333
INFO - haystack.modeling.evaluation.eval -  f1: 0.44
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.6826666666666666
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.32
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9600    0.9600    0.9600       350
     positive     0.4400    0.4400    0.4400        25

     accuracy                         0.9253       375
    macro avg     0.7000    0.7000    0.7000       375
 weighted avg     0.9253    0.9253    0.9253       375

Train epoch 3/4 (Cur. train loss: 0.4934):  71%|▋| 15/21 [02:38<00:56,  9.44s/it
Evaluating: 100%|█████████████████████████████████|

INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9600    0.9600    0.9600       350
     positive     0.4400    0.4400    0.4400        25

     accuracy                         0.9253       375
    macro avg     0.7000    0.7000    0.7000       375
 weighted avg     0.9253    0.9253    0.9253       375

Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.30s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 93 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation

In [ ]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_2"

In [ ]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=1,
    batch_size=8,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=40,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
    optimizer_name = "SGD",
    learning_rate = 1e-6
)

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_3"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=1,
    batch_size=4,
    grad_acc_steps=1, #aqui
    save_dir=save_dir,
    evaluate_every=4, #aqui
    embed_title=True,
    num_positives=1,
    num_hard_negatives=1,
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /w\   /w\   /w\   /|\  /w\   /|\  /w\   /|\  /|\  /w\   /w\   /|\  /w\   /w\   /w\ 
INFO - haystack.modeling.data_handler.data_silo -  / \   / \   / \   /'\   /'\   /

INFO - haystack.modeling.evaluation.eval -  average_rank: 1.36
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.8994    0.8994    0.8994       169
     positive     0.3200    0.3200    0.3200        25

     accuracy                         0.8247       194
    macro avg     0.6097    0.6097    0.6097       194
 weighted avg     0.8247    0.8247    0.8247       194

Train epoch 0/0 (Cur. train loss: 2.4617):  62%|▌| 16/26 [01:24<00:47,  4.74s/it
Evaluating: 100%|█████████████████████████████████| 7/7 [00:07<00:00,  1.03s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 16 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_4"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=3,
    batch_size=6,
    grad_acc_steps=1, #aqui
    save_dir=save_dir,
    evaluate_every=4, #aqui
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /|\  /|\  /w\   /|\  /w\   /w\   /w\   /w\   /|\  /w\   /w\   /|\  /|\  /w\   /w\   /w\ 
INFO - haystack.modeling.data_handler.data_silo -  /'\   /'\   / \   /'\   /'\   / 

INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8988505747126436
INFO - haystack.modeling.evaluation.eval -  f1: 0.12
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5094252873563219
INFO - haystack.modeling.evaluation.eval -  average_rank: 2.6
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9463    0.9463    0.9463       410
     positive     0.1200    0.1200    0.1200        25

     accuracy                         0.8989       435
    macro avg     0.5332    0.5332    0.5332       435
 weighted avg     0.8989    0.8989    0.8989       435

Train epoch 0/2 (Cur. train loss: 8.3217):  94%|▉| 16/17 [02:48<00:09,  9.61s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.35s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^

Train epoch 1/2 (Cur. train loss: 6.7013): 100%|█| 17/17 [03:00<00:00, 10.62s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.24s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 36 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.5681449270248413
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.9126436781609195
INFO - haystack.modeling.evaluation.eval -  f1: 0.24
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5763218390804598
INFO 

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_5"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=2,
    batch_size=4,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=6,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
    optimizer_name = "SGD"
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\   /|\  /w\   /|\  /w\   /w\   /|\  /w\ 
INFO - haystack.modeling.data_handler.data_silo -  / \   /'\   / \   / \   /'\  

INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8762886597938144
INFO - haystack.modeling.evaluation.eval -  f1: 0.28
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5781443298969072
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.4
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9323    0.9323    0.9323       266
     positive     0.2800    0.2800    0.2800        25

     accuracy                         0.8763       291
    macro avg     0.6062    0.6062    0.6062       291
 weighted avg     0.8763    0.8763    0.8763       291

Train epoch 0/1 (Cur. train loss: 9.6839):  92%|▉| 24/26 [03:40<00:11,  5.73s/it
Evaluating: 100%|█████████████████████████████████| 7/7 [00:11<00:00,  1.62s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^

Train epoch 1/1 (Cur. train loss: 2.7085): 100%|█| 26/26 [02:52<00:00,  6.64s/it
Evaluating: 100%|█████████████████████████████████| 7/7 [00:11<00:00,  1.65s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | TEST SET | AFTER 52 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 1.8074664306640624
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8831615120274914
INFO - haystack.modeling.evaluation.eval -  f1: 0.32
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.6015807560137457
INFO

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_6"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=2,
    batch_size=5,
    grad_acc_steps=2,
    save_dir=save_dir,
    evaluate_every=6,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=1,
    optimizer_name = "AdamW"
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /|\  /w\   /|\  /w\   /|\  /w\   /w\   /w\   /|\  /w\   /w\   /w\   /|\  /w\   /|\
INFO - haystack.modeling.data_handler.data_silo -  /'\   /'\   /'\   /'\   /'\   /'

INFO - haystack.modeling.evaluation.eval -  average_rank: 1.88
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9111    0.9111    0.9111       225
     positive     0.2000    0.2000    0.2000        25

     accuracy                         0.8400       250
    macro avg     0.5556    0.5556    0.5556       250
 weighted avg     0.8400    0.8400    0.8400       250

Train epoch 0/1 (Cur. train loss: 10.0916): 100%|█| 21/21 [01:59<00:00,  5.71s/i
Evaluating: 100%|█████████████████████████████████| 5/5 [00:08<00:00,  1.66s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 24 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_7"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=1,
    batch_size=6,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=2,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
    optimizer_name = "Adagrad"
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /|\  /w\   /w\   /w\   /|\  /w\   /w\   /|\  /w\   /w\   /w\   /w\   /|\  /w\   /w\   /|\
INFO - haystack.modeling.data_handler.data_silo -  /'\   / \   / \   / \   /'\   /

INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.903448275862069
INFO - haystack.modeling.evaluation.eval -  f1: 0.16
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5317241379310345
INFO - haystack.modeling.evaluation.eval -  average_rank: 3.0
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9488    0.9488    0.9488       410
     positive     0.1600    0.1600    0.1600        25

     accuracy                         0.9034       435
    macro avg     0.5544    0.5544    0.5544       435
 weighted avg     0.9034    0.9034    0.9034       435

Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.27s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************


Train epoch 0/0 (Cur. train loss: 12.0610): 100%|█| 17/17 [04:33<00:00, 16.11s/i
Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.22s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | TEST SET | AFTER 17 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 1.2742251968383789
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8988505747126436
INFO - haystack.modeling.evaluation.eval -  f1: 0.12
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5094252873563219
INFO

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_8"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=5,
    batch_size=6,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=3,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=1,
    optimizer_name = "RAdam"
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /w\   /w\   /w\   /|\  /w\   /|\  /w\   /w\   /w\   /w\   /w\   /|\  /w\   /w\   /w\ 
INFO - haystack.modeling.data_handler.data_silo -  / \   /'\   / \   /'\   /'\  

INFO - haystack.modeling.evaluation.eval -  average_rank: 2.32
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9208    0.9208    0.9208       265
     positive     0.1600    0.1600    0.1600        25

     accuracy                         0.8552       290
    macro avg     0.5404    0.5404    0.5404       290
 weighted avg     0.8552    0.8552    0.8552       290

Train epoch 0/4 (Cur. train loss: 6.6447):  71%|▋| 12/17 [01:33<00:37,  7.47s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:07<00:00,  1.59s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 12 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.9448779582975385
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8551724137931035
INFO - haystack.modeling.evaluation.eval -  f1: 0.16
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5075862068965518
INFO - haystack.modeling.evaluation.eval -  average_rank: 2.28
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9208    0.9208    0.9208       265
     positive     0.1600    0.1600    0.1600        25

     accuracy                         0.8552       290
    macro avg     0.5404    0.5404    0.5404       290
 weighted avg     0.8552    0.8552    0.8552       290

Train epoch 1/4 (Cur. train loss: 4.8498):  76%|▊| 13/17 [01:48<00:30,  7.74s/it
Evaluating: 100%|█████████████████████████████████|

Train epoch 2/4 (Cur. train loss: 3.6189):  65%|▋| 11/17 [01:26<00:44,  7.34s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:07<00:00,  1.42s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 45 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.8226884222030637
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8551724137931035
INFO - haystack.modeling.evaluation.eval -  f1: 0.16
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5075862068965518
INFO 

INFO - haystack.modeling.evaluation.eval -  f1: 0.24
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5544827586206896
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.96
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9283    0.9283    0.9283       265
     positive     0.2400    0.2400    0.2400        25

     accuracy                         0.8690       290
    macro avg     0.5842    0.5842    0.5842       290
 weighted avg     0.8690    0.8690    0.8690       290

Train epoch 3/4 (Cur. train loss: 5.8973):  71%|▋| 12/17 [01:36<00:36,  7.38s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:07<00:00,  1.47s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 63 BATCHES *****
****

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.5714811611175538
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8758620689655172
INFO - haystack.modeling.evaluation.eval -  f1: 0.28
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5779310344827586
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.84
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9321    0.9321    0.9321       265
     positive     0.2800    0.2800    0.2800        25

     accuracy                         0.8759       290
    macro avg     0.6060    0.6060    0.6060       290
 weighted avg     0.8759    0.8759    0.8759       290

Train epoch 4/4 (Cur. train loss: 2.9629):  76%|▊| 13/17 [01:43<00:29,  7.45s/it
Evaluating: 100%|█████████████████████████████████|

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_9"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=2,
    batch_size=6,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=5,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
    optimizer_name = "SGD",
    learning_rate = 1e-6
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /|\  /w\   /w\   /w\   /|\  /w\   /|\  /w\   /w\   /w\   /|\  /w\   /|\  /|\  /w\   /|\
INFO - haystack.modeling.data_handler.data_silo -  /'\   / \   / \   / \   /'\   / \

INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.903448275862069
INFO - haystack.modeling.evaluation.eval -  f1: 0.16
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5317241379310345
INFO - haystack.modeling.evaluation.eval -  average_rank: 3.32
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9488    0.9488    0.9488       410
     positive     0.1600    0.1600    0.1600        25

     accuracy                         0.9034       435
    macro avg     0.5544    0.5544    0.5544       435
 weighted avg     0.9034    0.9034    0.9034       435

Train epoch 0/1 (Cur. train loss: 9.8199): 100%|█| 17/17 [02:43<00:00,  9.62s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.21s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_10"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=3,
    batch_size=6,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=4,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
    optimizer_name = "SGD",
    learning_rate = 1e-7
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /|\  /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\ 
INFO - haystack.modeling.data_handler.data_silo -  /'\   / \   / \   /'\   /'\

INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.9080459770114943
INFO - haystack.modeling.evaluation.eval -  f1: 0.20000000000000004
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5540229885057472
INFO - haystack.modeling.evaluation.eval -  average_rank: 2.52
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9512    0.9512    0.9512       410
     positive     0.2000    0.2000    0.2000        25

     accuracy                         0.9080       435
    macro avg     0.5756    0.5756    0.5756       435
 weighted avg     0.9080    0.9080    0.9080       435

Train epoch 0/2 (Cur. train loss: 7.8085):  94%|▉| 16/17 [02:42<00:09,  9.37s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.27s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^

Train epoch 1/2 (Cur. train loss: 11.7614): 100%|█| 17/17 [02:59<00:00, 10.58s/i
Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.36s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 36 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.9985551452636718
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.9080459770114943
INFO - haystack.modeling.evaluation.eval -  f1: 0.20000000000000004
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5540229